In [1]:
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import time
from datetime import date
import statsmodels.formula.api as sm

#returns dataframe of price with close price
def ticker_df(ticker, start, end):
    source = 'yahoo'
    df = web.DataReader(ticker, source, start, end)
    return df.drop(['High', 'Low', 'Open', 'Adj Close', 'Volume'], axis = 1)

def stock_correlation(stock1, stock2):
    correlation = stock1['Close'].pct_change().corr(stock2['Close'].pct_change())
    return(correlation)

def get_correlation(stocks, start, end):
    
    assets ={}
    assets2 = {}

    for stock in stocks:
        assets[stock] = ticker_df(stock, start, end)

    for stock in stocks:
        assets2[stock] = list(assets[stock]['Close'])

    df = pd.DataFrame(data = assets2, columns = assets.keys())
    corr_matrix = df.corr()

    for stock1 in stocks:
        for stock2 in stocks:
            corr_matrix[stock1][stock2] = stock_correlation(assets[stock1], assets[stock2])
    
    return(corr_matrix)

#takes holdings {'ticker':allocation}, returns simulation of portfolio
def sim_port(holdings, start, end):
    portfolio = ticker_df('SPY', start, end).drop('Close', axis =1)

    for stock in holdings.keys():
        portfolio[stock + ' close'] = ticker_df(stock, start, end)
        portfolio[stock + ' pct change'] = portfolio[stock + ' close'].pct_change()

    for stock in holdings.keys():

        portfolio[stock + ' sim close'] = [0.0]*len(portfolio)
        portfolio[stock + ' sim close'][0] = holdings[stock]

        for i in range(1, len(portfolio)):
            a = portfolio[stock + ' sim close'][i-1]
            b = 1 + portfolio[stock + ' pct change'][i]
            portfolio[stock + ' sim close'][i] =  a*b

    portfolio['Close'] = [0.0]*len(portfolio)

    for stock in holdings.keys():
        for i in range(len(portfolio)):
            portfolio['Close'][i] += portfolio[stock + ' sim close'][i]

    return(portfolio)

#takes list of stocks (strings), returns dictionary of time series
def asset_dict(stocks, start, end):
    assets = {}
    for stock in stocks:
        assets[stock] = ticker_df(stock,start,end)['Close']
    return(assets)

#takes list of stocks, returns dictionary of time series
def asset_timeseries(stocks, start, end):
    assets = {}
    for stock in stocks:
        assets[stock] = list(ticker_df(stock, start, end)['Close'])
    return(assets)


def series_corr(ser1, ser2):
    correlation = ser1.pct_change().corr(ser2.pct_change())
    return(correlation)

def create_corr_matrix(stock_dict):
    data = pd.DataFrame(data = stock_dict, columns = stock_dict.keys())
    corr_matrix = data.corr()

    for stock1 in stock_dict.keys():
        for stock2 in stock_dict.keys():
            corr_matrix[stock1][stock2] = series_corr(stock_dict[stock1], stock_dict[stock2])
    return(corr_matrix)

#sharpe ratio = expected return daily - riskfreerate / stddev
def sharpe_ratio(asset, riskfree_rate):
    Rp = asset['Close'].pct_change().mean()
    Rff = riskfree_rate/252 
    sigma = asset['Close'].pct_change().std()
    ratio = (Rp-Rff)/sigma
    return(ratio)


In [22]:
start = dt.datetime(2007,1,1)
end = dt.datetime(2020,6,30)

IWM = ticker_df('IWM', start, end)
SPY = ticker_df('SPY', start, end)
QQQ = ticker_df('QQQ', start, end)

holdings = {'VGT':0.174358, 'VTI':0.026624, 'GLD':0.140878, 'USO':0.012532, 'LQD':0.050989, 'IEF':0.45764}
stocks = ['SPY', 'IWM', 'QQQ', 'GDX', 'GDXJ', 'GLD', 'SLV', 'USO', 'XLK', 'XLE', 'XRT', 'XRT', 'XLU', 'XLF','XLI','XLV', 'XAR', 'ARKK', 'TLT', 'LQD', 'IEF']
assets = asset_dict(stocks, start, end)

portfolio = sim_port(holdings, start, end)
assets['portfolio'] = portfolio['Close']
corr_matrix = create_corr_matrix(assets)
corr_matrix

,SPY,IWM,QQQ,GDX,GDXJ,GLD,SLV,USO,XLK,XLE,...,XLU,XLF,XLI,XLV,XAR,ARKK,TLT,LQD,IEF,portfolio
SPY,1.000000,0.913114,0.924764,0.257657,0.224771,0.028146,0.219077,0.412780,0.926177,0.825043,...,0.705302,0.860949,0.923545,0.843246,0.789977,0.760716,-0.446262,0.144567,-0.439917,0.540621
IWM,0.913114,1.000000,0.859284,0.245019,0.235532,0.023772,0.205757,0.382283,0.837658,0.767898,...,0.597363,0.836882,0.881350,0.753613,0.791162,0.789719,-0.427819,0.104751,-0.425623,0.486413
QQQ,0.924764,0.859284,1.000000,0.218276,0.198610,0.006507,0.179534,0.345559,0.961011,0.708100,...,0.589713,0.742910,0.838507,0.789465,0.694376,0.797257,-0.414106,0.103431,-0.414850,0.585045
GDX,0.257657,0.245019,0.218276,1.000000,0.940294,0.749886,0.679767,0.282389,0.214075,0.373862,...,0.265515,0.150076,0.240578,0.167691,0.112754,0.077527,0.046938,0.151660,0.061557,0.531157
GDXJ,0.224771,0.235532,0.198610,0.940294,1.000000,0.733689,0.659256,0.220812,0.185469,0.287968,...,0.241460,0.157562,0.213734,0.164093,0.150963,0.123491,0.064253,0.225418,0.102380,0.504237
GLD,0.028146,0.023772,0.006507,0.749886,0.733689,1.000000,0.799115,0.187172,0.009676,0.146269,...,0.100862,-0.052522,0.029724,0.001922,0.004150,-0.021323,0.147678,0.111776,0.181682,0.592225
SLV,0.219077,0.205757,0.179534,0.679767,0.659256,0.799115,1.000000,0.311519,0.171191,0.311209,...,0.188152,0.104951,0.214071,0.161948,0.169477,0.181708,-0.007642,0.143298,0.023227,0.562989
USO,0.412780,0.382283,0.345559,0.282389,0.220812,0.187172,0.311519,1.000000,0.359028,0.630707,...,0.227909,0.316062,0.385594,0.268397,0.324820,0.263697,-0.273002,0.040333,-0.246003,0.299291
XLK,0.926177,0.837658,0.961011,0.214075,0.185469,0.009676,0.171191,0.359028,1.000000,0.716672,...,0.614718,0.746731,0.839000,0.760612,0.712012,0.778445,-0.406100,0.115388,-0.408025,0.609065
XLE,0.825043,0.767898,0.708100,0.373862,0.287968,0.146269,0.311209,0.630707,0.716672,1.000000,...,0.593911,0.671627,0.781397,0.636333,0.666797,0.569463,-0.392947,0.123939,-0.376632,0.479072


In [14]:
corr_matrix.to_csv('corr_matrix.csv')


In [19]:
start = dt.datetime(2005,1,1)
end = dt.datetime(2020,6,30)

fin_instr = ['CADUSD=X', 'GLD', 'SLV', 'USO','VTI']
fin_instr_ser = asset_dict(fin_instr, start, end)

corr_matrix_curr = create_corr_matrix(fin_instr_ser)
corr_matrix_curr

,CADUSD=X,GLD,SLV,USO,VTI
CADUSD=X,1.000000,0.184902,0.206140,0.239634,0.289892
GLD,0.184902,1.000000,0.805231,0.198830,0.042223
SLV,0.206140,0.805231,1.000000,0.313910,0.220169
USO,0.239634,0.198830,0.313910,1.000000,0.405425
VTI,0.289892,0.042223,0.220169,0.405425,1.000000


,SPY,IWM,QQQ,GDX,GDXJ,GLD,SLV,USO,XLK,XLE,XRT,XLU,XLF,XLI,XLV,XAR,ARKK,TLT,LQD,portfolio
SPY,1.000000,0.901847,0.935922,0.081083,0.140907,-0.056771,0.165960,0.378633,0.943215,0.773610,0.784478,0.625923,0.902025,0.926928,0.878083,0.848603,0.761728,-0.418516,0.184954,0.919144
IWM,0.901847,1.000000,0.817599,0.084537,0.138616,-0.057306,0.179606,0.357117,0.817076,0.767254,0.850378,0.510905,0.872527,0.881299,0.772601,0.868747,0.792791,-0.388749,0.184461,0.895084
QQQ,0.935922,0.817599,1.000000,0.051894,0.110603,-0.054475,0.144819,0.308289,0.977861,0.631612,0.696720,0.506058,0.760708,0.801708,0.829209,0.734721,0.797392,-0.373177,0.140136,0.891360
GDX,0.081083,0.084537,0.051894,1.000000,0.951425,0.755357,0.633158,0.148195,0.050723,0.171530,0.039304,0.207333,-0.006613,0.080245,0.056321,0.110923,0.074283,0.255688,0.258909,0.297382
GDXJ,0.140907,0.138616,0.110603,0.951425,1.000000,0.737030,0.627762,0.149633,0.107460,0.205583,0.092687,0.250332,0.056991,0.134201,0.106086,0.154597,0.121440,0.201869,0.287669,0.347450
GLD,-0.056771,-0.057306,-0.054475,0.755357,0.737030,1.000000,0.766109,0.007532,-0.047268,-0.015975,-0.083399,0.166693,-0.151811,-0.071835,-0.040891,-0.025383,-0.024167,0.342837,0.331516,0.111946
SLV,0.165960,0.179606,0.144819,0.633158,0.627762,0.766109,1.000000,0.190252,0.149658,0.211815,0.121558,0.202900,0.089141,0.161786,0.131524,0.187696,0.184817,0.162659,0.295608,0.317889
USO,0.378633,0.357117,0.308289,0.148195,0.149633,0.007532,0.190252,1.000000,0.321061,0.626340,0.280387,0.116741,0.346334,0.363764,0.251513,0.342842,0.267796,-0.224575,0.042200,0.374542
XLK,0.943215,0.817076,0.977861,0.050723,0.107460,-0.047268,0.149658,0.321061,1.000000,0.655206,0.692238,0.534804,0.784741,0.824845,0.805184,0.755098,0.778624,-0.377186,0.162610,0.885217
XLE,0.773610,0.767254,0.631612,0.171530,0.205583,-0.015975,0.211815,0.626340,0.655206,1.000000,0.648199,0.441470,0.761526,0.779056,0.606881,0.721107,0.575273,-0.338310,0.156726,0.735460


In [129]:
df_factors = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench')[0]

In [60]:
# fama french data https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
# tutorial on excel https://www.youtube.com/watch?v=b2bO23z7cwg
# tutorial (text) https://www.codingfinance.com/post/2019-07-01-analyze-ff-factor-python/
# tutorial (text) https://randlow.github.io/posts/finance-economics/asset-pricing-regression/

# https://www.codingfinance.com/
# https://www.codingfinance.com/post/2019-07-01-analyze-ff-factor-python, adjust later
# https://seekingalpha.com/article/2035813-a-look-inside-the-fama-french-3-factor-model 
# https://www.codingfinance.com/post/2018-04-25-portfolio-beta-py/
# https://www.codingfinance.com/post/2018-03-27-retirement-py/
# https://www.codingfinance.com/post/2018-04-20-portfolio-stats-py/

In [137]:
factor_df = web.DataReader("F-F_Research_Data_Factors_daily", "famafrench")[0]
factor_df = factor_df.apply(lambda x: x/ 100)

In [217]:
#fama french 3 factor regression

factor_df_last = factor_df.index[factor_df.shape[0] - 1].date()
end = factor_df_last
factor_df_first = factor_df.index[0].date()
start = factor_df_first

holdings= {'XSU.TO':25.0, 'XQQ.TO':25.0, 'VEF.TO':30.0, 'GDX': 10.0, 'ARKK':5.0, 'XAR':5.0}
#holdings = {'SQQQ':100.0}
portfolio = sim_port(holdings, start, end)

factor_df['portfolio'] = portfolio['Close'].pct_change()
factor_df = factor_df[1:]
factor_df.rename(columns={"Mkt-RF":"mkt_excess"}, inplace=True)
factor_df['portfolio_excess'] = factor_df['portfolio'] - factor_df['RF']

model = smf.formula.ols(formula = "portfolio_excess ~ mkt_excess + SMB + HML", data = factor_df).fit()
print(model.params)


Intercept     0.000008
mkt_excess    0.881157
SMB           0.282471
HML          -0.095653
dtype: float64


c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [15]:
# portfolio is a "custom portfolio" using the sim_port function
# holdings is a dictionary in the form {asset: capital allocation} of assets in the portfolio
# sums all standard deviations from each holding, returns standard deviations of holding * capital allocation of holding divided by sum of all capital allocations

def risk_allocation(portfolio, holdings):
    total_holdings = sum(holdings.values())
    risk_components = {}
    for holding in holdings.keys():
        risk_components[holding] = portfolio[holding + ' close'].pct_change().std() * holdings[holding]/total_holdings
    
    rel_risk_allocation = {}
    for holding in holdings.keys():
        rel_risk_allocation[holding] = risk_components[holding]/sum(risk_components.values())
          
    return(rel_risk_allocation)

# portfolio is either a "custom portfolio" simulated using sim_port function, or is an individual stock,
# rf_gain is the riskfree gain, I use 2% per year, a high interest savigns rate
# margin interest of 1.5% per year from interactive brokers
# leverage ratio = 1.5 means borrowing 50% of the value of investment. leverage ratio = 1 means no leverage

def sharpe_ratio(portfolio, rf_gain, margin_interest, leverage_ratio):
    
    daily_rf = rf_gain/252
    
    daily_margin_interest = margin_interest*(leverage_ratio-1)/365
    
    daily_portfolio_gain = (portfolio['Close'].pct_change().mean() - daily_margin_interest)
    daily_portfolio_risk = portfolio['Close'].pct_change().std()
   
    sharpe_ratio = np.sqrt(252)*(daily_portfolio_gain - daily_rf)/daily_portfolio_risk
    return(sharpe_ratio, risk)

In [ ]:
def risk_allocation(portfolio, holdings):
    total_holdings = sum(holdings.values())
    risk_components = {}
    for holding in holdings.keys():
        risk_components[holding] = portfolio[holding + ' close'].pct_change().std() * holdings[holding]/total_holdings
    
    rel_risk_allocation = {}
    for holding in holdings.keys():
        rel_risk_allocation[holding] = risk_components[holding]/sum(risk_components.values())
          
    return(rel_risk_allocation)

In [26]:
holdings= {'XSU.TO':25.0, 'XQQ.TO':25.0, 'VEF.TO':30.0, 'GDX': 10.0, 'ARKK':5.0, 'XAR':5.0}
portfolio = sim_port(holdings, start, end)

In [19]:
annual_margin_interest = 0.0
leverage_ratio = 1
annual_HISA_rate = 0.02
# need to look at historical risk free rate of return
start = dt.datetime(2010,12,31)
end = dt.datetime(2020,5,31)

holdings= {'SPY':100.0}
#holdings= {'XSU.TO':25.0, 'XQQ.TO':25.0, 'VEF.TO':30.0, 'GDX': 10.0, 'ARKK':5.0, 'XAR':5.0}
portfolio = sim_port(holdings, start, end)
print(sharpe_ratio(portfolio, annual_HISA_rate, annual_margin_interest, leverage_ratio))

(0.7932050602625974, 0.19394868816719085)


In [424]:
annual_margin_interest = 0.015
leverage_ratio = 1.5
annual_HISA_rate = 0.02

start = dt.datetime(2007,1,1)
end = dt.datetime(2020,4,30)

holdings= {'SPY':30.0, 'TLT':40.0, 'IEF':15.0, 'GLD':7.50, 'DBC':7.50}

#holdings= {'XSU.TO':25.0, 'XQQ.TO':25.0, 'VEF.TO':30.0, 'GDX': 10.0, 'ARKK':5.0, 'XAR':5.0}
portfolio = sim_port(holdings, start, end)
print(sharpe_ratio(portfolio, annual_HISA_rate, annual_margin_interest, leverage_ratio))

0.2607304835677276
